#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import sklearn
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,plot_roc_curve,log_loss,f1_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import seaborn as sn
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, Dropout,LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from keras.models import Sequential
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Importing the dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/COVID19/Covid-19 Dataset_Clinical Data/ISRmergedtrain+test.csv')

#Split dataset 

In [4]:
X=df.values[:,0:8]
y=df.values[:,8]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

#SMOTE

In [6]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0))) 
  

Before OverSampling, counts of label '1': 5861
Before OverSampling, counts of label '0': 63601 



In [7]:
# import SMOTE module from imblearn library 
# pip install imblearn (if you don't have imblearn in your system) 
from imblearn.over_sampling import SMOTE 
sm = SMOTE() 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel()) 
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 

After OverSampling, the shape of train_X: (127202, 8)
After OverSampling, the shape of train_y: (127202,) 

After OverSampling, counts of label '1': 63601
After OverSampling, counts of label '0': 63601


In [8]:
feat_labels=['AGE', 'GENDER', 'FEVER', 'COUGH', 'FATIGUE', 'PAINS',
       'NASAL_CONGESTION', 'SHORTNESS_OF_BREATH', 'RUNNY_NOSE', 'SORE THROAT',
       'DIARRHEA', 'CHILLS', 'HEADACHE', 'VOMITING', 'LIVES_IN_AFFECTED_AREA']

#Metrics

In [9]:
def calculate_sensitivity_specificity(y_test, y_pred_test):
    # Note: More parameters are defined than necessary. 
    # This would allow return of other measures other than sensitivity and specificity
    
    # Get true/false for whether a breach actually occurred
    actual_pos = y_test == 1
    actual_neg = y_test == 0
    
    # Get true and false test (true test match actual, false tests differ from actual)
    true_pos = (y_pred_test == 1) & (actual_pos)
    false_pos = (y_pred_test == 1) & (actual_neg)
    true_neg = (y_pred_test == 0) & (actual_neg)
    false_neg = (y_pred_test == 0) & (actual_pos)
    
    # Calculate accuracy
    accuracy = np.mean(y_pred_test == y_test)
    
    # Calculate sensitivity and specificity
    sensitivity = np.sum(true_pos) / np.sum(actual_pos)
    specificity = np.sum(true_neg) / np.sum(actual_neg)
    
    return sensitivity, specificity, accuracy

#SBS

In [10]:
# for sfs
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
#knn = KNeighborsClassifier(n_neighbors=3) # ml_algo used = knn
#clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf = AdaBoostClassifier(n_estimators=100,learning_rate=0.1)
sfs1 = SFS(clf,  
           k_features=6, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy', cv=0)
#after applying sfs fit the data:
sfs1.fit(X_train, y_train)
sfs1.k_feature_names_  


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   26.0s finished

[2021-02-21 15:40:03] Features: 7/6 -- score: 0.9456825314560479[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   22.0s finished

[2021-02-21 15:40:25] Features: 6/6 -- score: 0.9452794333592468

('1', '2', '3', '4', '6', '7')

#SBS+SMOTE+ADABOOST

In [11]:
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
abc1= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=8,min_samples_leaf=1, min_samples_split=2 ), learning_rate=0.1, n_estimators=1000)

# Train Adaboost Classifer
abc1.fit(X_train_res[:,feat_cols], y_train_res)
y_predadb = abc1.predict(X_test[:,feat_cols])
# accuracy_score(y_test,y_predadb)
p=precision_score(y_test, y_predadb, average='micro')
r=recall_score(y_test, y_predadb,average='micro')
f=f1_score(y_test, y_predadb,average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predadb)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))



[1, 2, 3, 4, 6, 7]
Accuracy:    92.52%
Sensitivity:   92.52%
Specificity:  93.58%
Precision:  92.52%
Recall:  92.52%
F1_score: 92.52%
AUC: 93.05%


#MLP Classifier

In [12]:
#Initializing the MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',solver='adam',random_state=13,validation_fraction=0.2)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

clf= MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',solver='adam',random_state=14,validation_fraction=0.2)
sfs0 = SFS(clf,  
           k_features=5, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy', cv=0)
           

#after applying sfs fit the data:
sfs0.fit(X_train, y_train)
sfs0.k_feature_names_  


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  3.8min finished

[2021-02-21 15:45:52] Features: 7/5 -- score: 0.9583369324234834[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  2.8min finished

[2021-02-21 15:48:41] Features: 6/5 -- score: 0.9573291871814805[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.1min finished

[2021-02-21 15:50:44] Features: 5/5 -- score: 0.9559759292850767

('0', '2', '3', '4', '7')

In [14]:
feat_cols = list(sfs0.k_feature_idx_)
print(feat_cols)
classifier= MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',alpha=1e-5,solver='adam',random_state=16)
classifier.fit(X_train_res[:,feat_cols], y_train_res)
y_predmlp1 = classifier.predict(X_test[:,feat_cols])
accuracy_score(y_test,y_predmlp1)
p=precision_score(y_test, y_predmlp1 , average='micro')
r=recall_score(y_test, y_predmlp1 , average='micro')
f=f1_score(y_test, y_predmlp1 , average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predmlp1)

print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))



[0, 2, 3, 4, 7]
Accuracy:    86.98%
Sensitivity:   86.98%
Specificity:  87.44%
Precision:  86.98%
Recall:  86.98%
F1_score: 86.98%
AUC: 87.21%


#XGBoost Classifier

In [15]:
from xgboost import XGBClassifier
clf = XGBClassifier(learning_rate=0.1,n_estimators=100, random_state=256)
sfs2 = SFS(clf,  
           k_features=5, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs2.fit(X_train, y_train)
sfs2.k_feature_names_  


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   15.0s finished

[2021-02-21 15:51:38] Features: 7/5 -- score: 0.9583369324234834[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   12.0s finished

[2021-02-21 15:51:50] Features: 6/5 -- score: 0.9573291871814805[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    9.5s finished

[2021-02-21 15:51:59] Features: 5/5 -- score: 0.9559759292850767

('0', '2', '3', '4', '7')

In [16]:
feat_cols = list(sfs2.k_feature_idx_)
print(feat_cols)

xgboost1 = XGBClassifier(max_depth=7,learning_rate=0.1,n_estimators=1000,random_state=178, subsample= 0.9,colsample_bytree=1,
gamma= 5)

xgboost1.fit(X_train_res[:,feat_cols], y_train_res)
# make predictions for test data
predxgboost1 = xgboost1.predict(X_test[:,feat_cols])
accuracy_score(y_test,predxgboost1)
p=precision_score(y_test, predxgboost1,average='micro')
r=recall_score(y_test, predxgboost1,average='micro')
f=f1_score(y_test, predxgboost1,average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, predxgboost1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[0, 2, 3, 4, 7]
Accuracy:    86.98%
Sensitivity:   86.98%
Specificity:  87.44%
Precision:  86.98%
Recall:  86.98%
F1_score: 86.98%
AUC: 87.21%


#SVM

In [17]:
from sklearn import svm
clf = svm.SVC(kernel='linear') # Linear Kernel
sfs3 = SFS(clf,  
           k_features=5, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs3.fit(X_train, y_train)
sfs3.k_feature_names_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.5min finished

[2021-02-21 15:54:35] Features: 7/5 -- score: 0.9559759292850767[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  1.3min finished

[2021-02-21 15:55:53] Features: 6/5 -- score: 0.9559759292850767[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.1min finished

[2021-02-21 15:57:00] Features: 5/5 -- score: 0.9559759292850767

('0', '2', '3', '4', '7')

In [18]:
feat_cols = list(sfs3.k_feature_idx_)
print(feat_cols)
SVM1 = svm.SVC(kernel='linear',C=0.1) # Linear Kernel
SVM1.fit(X_train_res[:, feat_cols], y_train_res)

#Predict the response for test dataset
y_predsvm1 = SVM1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predsvm1,average='micro')
r=recall_score(y_test, y_predsvm1,average='micro')
f=f1_score(y_test, y_predsvm1,average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predsvm1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[0, 2, 3, 4, 7]
Accuracy:    86.98%
Sensitivity:   86.98%
Specificity:  87.44%
Precision:  86.98%
Recall:  86.98%
F1_score: 86.98%
AUC: 87.21%


#Random Forest Classifier



In [19]:
clf=RandomForestClassifier(n_estimators=50,max_depth=4)

sfs4 = SFS(clf,  
           k_features=6, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs4.fit(X_train, y_train)
sfs4.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    6.0s finished

[2021-02-21 15:58:39] Features: 7/6 -- score: 0.956105496530477[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    5.1s finished

[2021-02-21 15:58:44] Features: 6/6 -- score: 0.9559759292850767

('0', '1', '2', '3', '4', '7')

In [20]:
feat_cols = list(sfs4.k_feature_idx_)
print(feat_cols)


[0, 1, 2, 3, 4, 7]


In [21]:

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
RFC1=RandomForestClassifier(n_estimators=500, max_depth=4)

#RFC1=RandomForestClassifier(n_estimators=2000,random_state=450)
#Train the model using the training sets y_pred=clf.predict(X_test)
RFC1.fit(X_train_res[:,feat_cols],y_train_res)
y_predrf1=RFC1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predrf1 , average='micro')
r=recall_score(y_test, y_predrf1 , average='micro')
f=f1_score(y_test, y_predrf1, average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predrf1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))


Accuracy:    92.52%
Sensitivity:   92.52%
Specificity:  93.58%
Precision:  92.52%
Recall:  92.52%
F1_score: 92.52%
AUC: 93.05%


#Logistic Regression

In [22]:
clf= LogisticRegression()

sfs5 = SFS(clf,  
           k_features=5, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs5.fit(X_train, y_train)
sfs5.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.9s finished

[2021-02-21 15:58:56] Features: 7/5 -- score: 0.9528087299530679[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.6s finished

[2021-02-21 15:58:58] Features: 6/5 -- score: 0.9559759292850767[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.4s finished

[2021-02-21 15:58:59] Features: 5/5 -- score: 0.9559759292850767

('2', '3', '4', '5', '7')

In [23]:
feat_cols = list(sfs5.k_feature_idx_)
lr1 = LogisticRegression(random_state=999)
lr1.fit(X_train_res[:,feat_cols],y_train_res)
y_predlr1=lr1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predlr1 , average='micro')
r=recall_score(y_test, y_predlr1,average='micro')
f=f1_score(y_test, y_predlr1, average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predlr1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

Accuracy:    95.56%
Sensitivity:   95.56%
Specificity:  98.20%
Precision:  95.56%
Recall:  95.56%
F1_score: 95.56%
AUC: 96.88%



#Decision Tree classifier

In [24]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
clf = DecisionTreeClassifier()
sfs7 = SFS(clf,  
           k_features=5, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2, 
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs7.fit(X_train, y_train)
sfs7.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s finished

[2021-02-21 15:59:00] Features: 7/5 -- score: 0.9583369324234834[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s finished

[2021-02-21 15:59:00] Features: 6/5 -- score: 0.9573291871814805[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s finished

[2021-02-21 15:59:00] Features: 5/5 -- score: 0.9559759292850767

('0', '2', '3', '4', '7')

In [25]:
feat_cols = list(sfs7.k_feature_idx_)
print(feat_cols)
dtc1 = DecisionTreeClassifier(random_state=0, max_depth=4)
dtc1.fit(X_train_res[:,feat_cols],y_train_res)
pred_dtc1=dtc1.predict(X_test[:,feat_cols])
p=precision_score(y_test, pred_dtc1 , average='micro')
r=recall_score(y_test, pred_dtc1 ,average='micro')
f=f1_score(y_test, pred_dtc1 , average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, pred_dtc1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[0, 2, 3, 4, 7]
Accuracy:    86.89%
Sensitivity:   86.89%
Specificity:  87.45%
Precision:  86.89%
Recall:  86.89%
F1_score: 86.89%
AUC: 87.17%


# Gradient Boosting

In [26]:

clf = GradientBoostingClassifier(learning_rate=0.001,n_estimators=50,random_state=6)

sfs8 = SFS(clf,  
           k_features=5, 
           forward=False, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs8.fit(X_train, y_train)
sfs8.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    7.6s finished

[2021-02-21 15:59:09] Features: 7/5 -- score: 0.9156229305231638[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    6.2s finished

[2021-02-21 15:59:15] Features: 6/5 -- score: 0.9156229305231638[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    4.7s finished

[2021-02-21 15:59:20] Features: 5/5 -- score: 0.9156229305231638

('0', '1', '2', '3', '4')

In [27]:
feat_cols = list(sfs8.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 4]


In [28]:

GBC11=GradientBoostingClassifier(learning_rate=0.001,n_estimators=1000,max_depth=4)
GBC11.fit(X_train_res[:,feat_cols], y_train_res)
y_test_pred11 = GBC11.predict(X_test[:,feat_cols])
#accuracy_score(y_test,y_test_pred)
p=precision_score(y_test, y_test_pred11 , average='micro')
r=recall_score(y_test, y_test_pred11 , average='micro')
f=f1_score(y_test, y_test_pred11, average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_test_pred11)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))


Accuracy:    85.75%
Sensitivity:   85.75%
Specificity:  86.94%
Precision:  85.75%
Recall:  85.75%
F1_score: 85.75%
AUC: 86.35%
